In [ ]:
from fastapi import APIRouter, HTTPException, Depends
from sqlalchemy.orm import Session, joinedload
from typing import List
from models import Book, BookWriter, Writer
from schemas import BookDto, BookResponse
from database import get_db

router = APIRouter(prefix="/api/books", tags=["Books"])

# CREATE
@router.post("/", response_model=BookResponse)
def create_book(book_dto: BookDto, db: Session = Depends(get_db)):
    book = Book(
        book_code=book_dto.book_code,
        book_name=book_dto.book_name,
        isbn=book_dto.isbn,
        book_price=book_dto.book_price,
        date=book_dto.date,
        book_page=book_dto.book_page,
        book_description=book_dto.book_description,
        cover_image_path=book_dto.cover_image_path,
        mobile_no=book_dto.mobile_no,
        book_writers=[
            BookWriter(writer_id=writer_id) for writer_id in book_dto.writer_ids
        ]
    )
    db.add(book)
    db.commit()
    db.refresh(book)
    return book

# READ
@router.get("/", response_model=List[BookResponse])
def get_books(db: Session = Depends(get_db)):
    books = db.query(Book).options(joinedload(Book.book_writers).joinedload(BookWriter.writer)).all()
    return books

# UPDATE
@router.put("/{book_id}", response_model=BookResponse)
def update_book(book_id: int, book_dto: BookDto, db: Session = Depends(get_db)):
    book = db.query(Book).options(joinedload(Book.book_writers)).filter(Book.id == book_id).first()
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")

    book.book_code = book_dto.book_code
    book.book_name = book_dto.book_name
    book.isbn = book_dto.isbn
    book.book_price = book_dto.book_price
    book.date = book_dto.date
    book.book_page = book_dto.book_page
    book.book_description = book_dto.book_description
    book.cover_image_path = book_dto.cover_image_path
    book.mobile_no = book_dto.mobile_no

    # Clear and reassign BookWriters
    book.book_writers.clear()
    book.book_writers = [
        BookWriter(writer_id=writer_id, book_id=book.id)
        for writer_id in book_dto.writer_ids
    ]

    db.commit()
    db.refresh(book)
    return book

# DELETE
@router.delete("/{book_id}", status_code=204)
def delete_book(book_id: int, db: Session = Depends(get_db)):
    book = db.query(Book).filter(Book.id == book_id).first()
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")

    db.delete(book)
    db.commit()
    return
